In [1]:
import numpy as np
from qutip import *
from helpers.estimation import *
from helpers.data_processing import *
from helpers.targets import *
import exp_params
import scipy as sc

USING W-FUNCTION POINTS


### Check yourself: print function, dimension, and number of displacements

In [2]:
print(f"{exp_params.which_function}\nD={exp_params.D}\nnD={exp_params.nD}")

W
D=6
nD=35


### State list 

In [3]:
state_list = exp_params.fock_state_list
print(state_list)

['fock0' 'fock01' 'fock0i1' 'fock02' 'fock0i2' 'fock1' 'fock03' 'fock0i3'
 'fock12' 'fock1i2' 'fock04' 'fock0i4' 'fock13' 'fock1i3' 'fock2' 'fock05'
 'fock0i5' 'fock14' 'fock1i4' 'fock23' 'fock2i3' 'fock15' 'fock1i5'
 'fock24' 'fock2i4' 'fock3' 'fock25' 'fock2i5' 'fock34' 'fock3i4' 'fock35'
 'fock3i5' 'fock4' 'fock45' 'fock4i5' 'fock5']


### Choose whether to use grape or ideal for state preparation and target states, and specify target state directory

In [4]:
use_grape = False

# GRAPE generated target states
target_states_directory = r"target_states" 

### Gather linear inversion variables used to generate Least-Squares estimator, $\rho_{LS}$

In [5]:
# Load inverse map variables for converting pe to rho
map_variables = np.load(
    f"map_variables\map_variables_D={exp_params.D}_nD={exp_params.nD}_W.npz"
)
W = map_variables["W"]
beta = map_variables["beta"]


### Function for protocol simulation with QuTiP

In [6]:
def wigner_sim(rho_init):
    H_dis = exp_params.H0

    Hd = (
        2 * np.pi * exp_params.power_rabi_A_W * 1j * (exp_params.Qd - exp_params.Q) / 2
    )  # 1/2 factor for Ry(pi/2)
    H = [H_dis, [Hd, exp_params.pulse_W]]

    Hdm = (
        -2 * np.pi * exp_params.power_rabi_A_W * 1j * (exp_params.Qd - exp_params.Q) / 2
    )  # 1/2 factor for Ry(pi/2)
    Hm = [H_dis, [Hdm, exp_params.pulse_W]]

    tlist_wait = np.linspace(0, exp_params.twait_W, 11)
    tlist_pi_half = np.linspace(0, exp_params.sigma_W * exp_params.chop_W, 101)

    # first pi/2
    result1 = mesolve(H, rho_init, tlist_pi_half, c_ops=exp_params.c_ops_qc)

    # dispersive coupling
    result2 = mesolve(H_dis, result1.states[-1], tlist_wait, c_ops=exp_params.c_ops_qc)

    # second pi/2
    result3 = mesolve(H, result2.states[-1], tlist_pi_half, c_ops=exp_params.c_ops_qc)

    # second -pi/2
    result3m = mesolve(Hm, result2.states[-1], tlist_pi_half, c_ops=exp_params.c_ops_qc)

    rho_qt = result3.states[-1].ptrace([0])
    rho_qtm = result3m.states[-1].ptrace([0])

    # w_id[i] = (par * rho_c).tr().real
    pe = (rho_qt * (exp_params.ue * exp_params.ue.dag())).tr()
    pem = (rho_qtm * (exp_params.ue * exp_params.ue.dag())).tr()

    return np.around(pe, 6), np.around(pem, 6)

### Iterate through the states, run the protocol, and record in numpy arrays the fidelities, standard deviations, and $\rho_{BME}$ for each. This saves both the normal Wigner and corrected Wigner information.

In [7]:
F_MLE_corr = np.zeros(len(state_list), dtype=float)
F_Bayes_BME_corr = np.zeros(len(state_list), dtype=float)

F_MLE_norm = np.zeros(len(state_list), dtype=float)
F_Bayes_BME_norm = np.zeros(len(state_list), dtype=float)

In [8]:
for j, state_name in enumerate(state_list):
    if use_grape:
        rho_tar_qc = Y_target(
            state_name, target_states_directory, exp_params.qdim, exp_params.cdim
        )
    else:
        rho_tar_qc = tensor(
            fock_dm(exp_params.qdim, 0),
            ket2dm(cavity_target_state(state_name, exp_params.cdim)),
        )

    Pid_corr = np.zeros(len(exp_params.disp_points))

    Pid_norm = np.zeros(len(exp_params.disp_points))

    if use_grape:
        if state_name in exp_params.pes_after_grape_sim[:, 0]:
            pge = exp_params.pes_after_grape_sim[
                (exp_params.pes_after_grape_sim[:, 0] == state_name), 1
            ].astype(float)

        else:
            pge = exp_params.pe_after_grape_sim_avg
    else:
        pge = 0

    for w in range(len(exp_params.disp_points)):
        RHO_tar = (
            tensor(
                qeye(exp_params.qdim),
                displace(exp_params.cdim, exp_params.disp_points[w]),
            )
            * rho_tar_qc
            * tensor(
                qeye(exp_params.qdim),
                displace(exp_params.cdim, -exp_params.disp_points[w]),
            )
        )
        pe, pem = wigner_sim(RHO_tar)

        Pid_corr[w] = (
            (pe - pge) / (1 - 2 * pge) - (pem - pge) / (1 - 2 * pge)
        ).squeeze()

        Pid_norm[w] = (2 * (pe - pge) / (1 - 2 * pge) - 1).squeeze()
    ##################################################################


    rho_est_corr, qRho_est_corr = get_LS_and_MLE_rho_est(
        Pid_corr, W, beta, exp_params.D, exp_params.nD
    )
    rho_est_norm, qRho_est_norm = get_LS_and_MLE_rho_est(
        Pid_norm, W, beta, exp_params.D, exp_params.nD
    )


    rho_tar_c = Qobj(rho_tar_qc.ptrace(1)[0 : exp_params.D, 0 : exp_params.D])

    rho_tar_c = rho_tar_c / rho_tar_c.tr()  # normalise it, .unit()


    Fmean_bayes_corr, Fstd_bayes_corr, rho_BME_corr = bayesian_rho_est(

        numSamp=2**10, N=exp_params.N_sim, rho_tar=rho_tar_c, rhoLS=rho_est_corr.full()
    )

    Fmean_bayes_norm, Fstd_bayes_norm, rho_BME_norm = bayesian_rho_est(
        numSamp=2**10, N=exp_params.N_sim, rho_tar=rho_tar_c, rhoLS=rho_est_norm.full()
    )

    F_Bayes_BME_corr[j] = fidelity(rho_tar_c, Qobj(rho_BME_corr)) ** 2
    F_Bayes_BME_norm[j] = fidelity(rho_tar_c, Qobj(rho_BME_norm)) ** 2

    F_MLE_corr[j] = fidelity(rho_tar_c, qRho_est_corr) ** 2
    F_MLE_norm[j] = fidelity(rho_tar_c, qRho_est_norm) ** 2

    print(f"state {j}: {state_name}, corr: {F_Bayes_BME_corr[j]}")
    print(f"state {j}: {state_name}, norm: {F_Bayes_BME_norm[j]}\n")

state 0: fock0, corr: 0.973733569094973
state 0: fock0, norm: 0.9152341013062958



### Save all the data so that we don't have to run this simulation again

In [9]:
print("CORRECTED WIGNER")
print("MLE Fidelity: \n", np.dstack((state_list, F_MLE_corr)))
print(f"Average MLE fidelity is {np.mean(F_MLE_corr)}")
print("\nBayes Fidelity:\n", np.dstack((state_list, F_Bayes_BME_corr)))
print(f"Average Bayes fidelity is {np.mean(F_Bayes_BME_corr)}")

print("\nNORMAL WIGNER")
print("MLE Fidelity: \n", np.dstack((state_list, F_MLE_norm)))
print(f"Average MLE fidelity is {np.mean(F_MLE_norm)}")
print("\nBayes Fidelity:\n", np.dstack((state_list, F_Bayes_BME_norm)))
print(f"Average Bayes fidelity is {np.mean(F_Bayes_BME_norm)}")


if use_grape:
    np.savez(
        f"results_dimensions\sim\W\grape_target_states\Bayes_F_D={exp_params.D}_nD={exp_params.nD}_grape_sim_W.npz",
        F_MLE_corr=F_MLE_corr,
        F_Bayes_BME_corr=F_Bayes_BME_corr,
        F_MLE_norm=F_MLE_norm,
        F_Bayes_BME_norm=F_Bayes_BME_norm,
    )
else:
    np.savez(
        f"results_dimensions\sim\W\ideal_target_states\Bayes_F_D={exp_params.D}_nD={exp_params.nD}_ideal_sim_W.npz",
        F_MLE_corr=F_MLE_corr,
        F_Bayes_BME_corr=F_Bayes_BME_corr,
        F_MLE_norm=F_MLE_norm,
        F_Bayes_BME_norm=F_Bayes_BME_norm,
    )

CORRECTED WIGNER
MLE Fidelity: 
 [[['fock0' '0.9965132445839805']
  ['fock01' '0.9927073997212734']
  ['fock0i1' '0.9996023200287264']
  ['fock1' '0.976729430867074']]]
Average MLE fidelity is 0.9913880988002637

Bayes Fidelity:
 [[['fock0' '0.9964309958024536']
  ['fock01' '0.9934093708296619']
  ['fock0i1' '0.999587963595529']
  ['fock1' '0.984313237190734']]]
Average Bayes fidelity is 0.9934353918545946

NORMAL WIGNER
MLE Fidelity: 
 [[['fock0' '0.996281096279515']
  ['fock01' '0.992633351659532']
  ['fock0i1' '0.9995617335072582']
  ['fock1' '0.9766957702712872']]]
Average MLE fidelity is 0.9912929879293981

Bayes Fidelity:
 [[['fock0' '0.9962298466628889']
  ['fock01' '0.9927248879465185']
  ['fock0i1' '0.9995448024307142']
  ['fock1' '0.9793575813875482']]]
Average Bayes fidelity is 0.9919642796069175
